In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import random
import datasets 
import matplotlib.pyplot as plt
from tqdm import tqdm
from datasets import concatenate_datasets
from rdkit import Chem, rdBase

from mol_depict_cdk.cxsmiles_tokenizer import CXSMILESTokenizer

/mnt/volume/anaconda3/envs/unidoc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Clean Mixtral augmentations

In [21]:
# Read for dataloader

dataset_name_1 = "mdu_3005_aug"
dataset_hf_1 = concatenate_datasets([
    datasets.load_from_disk(os.getcwd()  + f"/../../../deepsearch-ai-unidoc/data/{dataset_name_1}/", keep_in_memory=False)["train"],
    datasets.load_from_disk(os.getcwd() + f"/../../../deepsearch-ai-unidoc/data/{dataset_name_1}/", keep_in_memory=False)["test"]
])
dataset_name_2 = "mdu_3006_aug"
dataset_hf_2 = concatenate_datasets([
    datasets.load_from_disk(os.getcwd()  + f"/../../../deepsearch-ai-unidoc/data/{dataset_name_2}/", keep_in_memory=False)["train"],
    datasets.load_from_disk(os.getcwd() + f"/../../../deepsearch-ai-unidoc/data/{dataset_name_2}/", keep_in_memory=False)["test"]
])
dataset_hf = concatenate_datasets([dataset_hf_1, dataset_hf_2])
dataset_hf = dataset_hf.shuffle()
print(dataset_hf)

Dataset({
    features: ['id', 'page_image_path', 'description', 'annotation', 'mol', 'cxsmiles_dataset', 'cxsmiles', 'cxsmiles_opt', 'keypoints', 'cells', 'page_image'],
    num_rows: 20000
})


In [14]:
i_max = 5000000000000 #float("inf")

remove_indices = []
cxsmiles_tokenizer = CXSMILESTokenizer()
for i, description in tqdm(enumerate(dataset_hf["description"]), total=min(i_max, len(dataset_hf))):
    if i > i_max:
        break
    # display(page_image.resize((512, 512)))
    # plt.show()
    if ("ictionnary" in description) or ("ictionary" in description):
        remove_indices.append(i)
        
print(remove_indices)

100%|██████████| 295570/295570 [00:00<00:00, 1022594.25it/s]

[128, 159, 282, 304, 474, 753, 768, 848, 888, 893, 1314, 1364, 1476, 1735, 1741, 2133, 2242, 2439, 2638, 2707, 3237, 3269, 3396, 3509, 3573, 3590, 3628, 3948, 4106, 4302, 4317, 4410, 4455, 4643, 5168, 5310, 5526, 5595, 5794, 5803, 5932, 6577, 6725, 6859, 7148, 7404, 7482, 7527, 7670, 7756, 7848, 8350, 8357, 8434, 8489, 8514, 8611, 8761, 8772, 8827, 9015, 9127, 9484, 9534, 9579, 9608, 9880, 10315, 10458, 10626, 10782, 11165, 11219, 11237, 11258, 11358, 11538, 11641, 11864, 11869, 11934, 12027, 12076, 12413, 12569, 12649, 13014, 13156, 13208, 13226, 13667, 13682, 13778, 13787, 13998, 14073, 14093, 14268, 14288, 14337, 14375, 14405, 14594, 14665, 14761, 15145, 15180, 15481, 15718, 15768, 15873, 16366, 16586, 16762, 16784, 16910, 16934, 17244, 17404, 17560, 17774, 17809, 17851, 17883, 17902, 18015, 18146, 18319, 18568, 18691, 18875, 19138, 19320, 19334, 20187, 20215, 20268, 20285, 20405, 20499, 20525, 20605, 20624, 21174, 21254, 21307, 21499, 21729, 21838, 22310, 22490, 23635, 23729, 23908

In [15]:
dataset_hf = dataset_hf.select((i for i in range(len(dataset_hf)) if i not in set(remove_indices)))

Parameter 'indices'=<generator object <genexpr> at 0x7f03bde77a00> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [17]:
hf_dataset_name = "mdu_3012_aug"
dataset_hf = dataset_hf.train_test_split(test_size=0.1)
dataset_hf.save_to_disk(os.getcwd() + f"/../../../deepsearch-ai-unidoc/data/{hf_dataset_name}/")

Saving the dataset (8/8 shards): 100%|██████████| 29373/29373 [00:28<00:00, 1046.06 examples/s]


### Collate Augmentated Dataset with Existing Dataset

In [13]:
# Read for dataloader

dataset_name_1 = "mdu_3008_aug"
dataset_hf_1 = concatenate_datasets([
    datasets.load_from_disk(os.getcwd()  + f"/../../../deepsearch-ai-unidoc/data/{dataset_name_1}/", keep_in_memory=False)["train"],
    datasets.load_from_disk(os.getcwd() + f"/../../../deepsearch-ai-unidoc/data/{dataset_name_1}/", keep_in_memory=False)["test"]
])
dataset_name_2 = "mdu_3010_aug"
dataset_hf_2 = concatenate_datasets([
    datasets.load_from_disk(os.getcwd()  + f"/../../../deepsearch-ai-unidoc/data/{dataset_name_2}/", keep_in_memory=False)["train"],
    datasets.load_from_disk(os.getcwd() + f"/../../../deepsearch-ai-unidoc/data/{dataset_name_2}/", keep_in_memory=False)["test"]
])
dataset_name_3 = "mdu_3011_aug"
dataset_hf_3 = concatenate_datasets([
    datasets.load_from_disk(os.getcwd()  + f"/../../../deepsearch-ai-unidoc/data/{dataset_name_3}/", keep_in_memory=False)["train"],
    datasets.load_from_disk(os.getcwd() + f"/../../../deepsearch-ai-unidoc/data/{dataset_name_3}/", keep_in_memory=False)["test"]
])
dataset_hf = concatenate_datasets([dataset_hf_1, dataset_hf_2, dataset_hf_3])
dataset_hf = dataset_hf.shuffle()
print(dataset_hf)

Dataset({
    features: ['id', 'page_image_path', 'description', 'annotation', 'mol', 'cxsmiles_dataset', 'cxsmiles', 'cxsmiles_opt', 'keypoints', 'cells', 'page_image'],
    num_rows: 295570
})


In [ ]:
hf_dataset_name = "mdu_3012_aug"
dataset_hf = dataset_hf.train_test_split(test_size=0.1)
dataset_hf.save_to_disk(os.getcwd() + f"/../../../deepsearch-ai-unidoc/data/{hf_dataset_name}/")